In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import pandas as pd
import multiprocessing as mp
import sys
import threading
import re
import numpy as np

In [67]:
def get_speeches(url1, url2, letter, return_dict):
    if letter == 'c':
        r = requests.get(url2.format(letter))
        soup = BeautifulSoup(r.text, 'html.parser')
        content_list = soup.find('div', attrs={'class':'site-content'})
    else:
        r = requests.get(url1.format(letter))
        soup = BeautifulSoup(r.text, 'html.parser')
        content_list = soup.find('div', attrs={'class':'entry-content'})
    speeches = []
    for result in tqdm_notebook(content_list.findAll('a', href=True)):
        if result.text != 'Speeches Index':
            try:
                speech_url = result.get('href')
                speech_r = requests.get(speech_url)
                speech_soup = BeautifulSoup(speech_r.text, 'html.parser')
                speech_text = speech_soup.find('div', attrs={'class':'entry-content'}).text
                speeches.append([result.text,speech_text])
            except:
                try:
                    speeches.append([result.text,'unable to scrape'])
                except:
                    pass
    return_dict[letter]=speeches

    
def request_thread(url1, url2, letters):
    manager = mp.Manager()
    return_dict = manager.dict()
    threads = []
    for letter in tqdm_notebook(letters):
        thread = threading.Thread(name=letter, 
                                  target=get_speeches, 
                                  args=(url1, url2, letter, return_dict))
        thread.start()
        threads.append(thread)

    for t in threads:
        t.join()

    return return_dict

In [14]:
from string import ascii_lowercase

In [57]:
url1 = "http://www.ukpol.co.uk/speeches/speeches-{}/"
url2 = "http://www.ukpol.co.uk/speeches/{}/"

In [27]:
my_letters = ascii_lowercase
for letter in my_letters:
    if letter == 'x':
        my_letters = my_letters.replace(letter,'')

In [68]:
speeches_dict = request_thread(url1, url2, my_letters)

Exception in thread j:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/managers.py", line 749, in _callmethod
    conn = self._tls.connection
AttributeError: 'ForkAwareLocal' object has no attribute 'connection'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-35-00fe20fb9716>", line 21, in get_speeches
    return_dict[letter]=speeches
  File "<string>", line 2, in __setitem__
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/managers.py", line 753, in _callmethod
    self._connect()
  File "/Library/Frameworks/Python.fr

Exception in thread m:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/managers.py", line 749, in _callmethod
    conn = self._tls.connection
AttributeError: 'ForkAwareLocal' object has no attribute 'connection'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-35-00fe20fb9716>", line 21, in get_speeches
    return_dict[letter]=speeches
  File "<string>", line 2, in __setitem__
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/managers.py", line 753, in _callmethod
    self._connect()
  File "/Library/Frameworks/Python.fr

In [60]:
speeches1 = pd.DataFrame([x for letter in speeches_dict.values() for x in letter])

In [61]:
speeches1.columns = ['title','speech']

In [62]:
speeches1.head()

,title,speech
0,"Isherwood, Mark – 2006 Speech on Fuel Poverty",\nBelow is the text of the speech made by Mark...
1,"Umunna, Chuka – 2011 Speech to Universities UK",\nBelow is the text of the speech made by Chuk...
2,"Umunna, Chuka – 2012 Speech to High Pay Commis...",\nBelow is the text of the speech made by Chuk...
3,"Umunna, Chuka – 2012 Speech to Hub Westminster",\nBelow is the text of the speech made by Chuk...
4,"Umunna, Chuka – 2012 Speech to UCATT Conference",\nBelow is the text of the speech made by Chuk...


In [63]:
speeches1.shape

(3077, 2)

In [4]:
speeches1.to_csv('speeches1.csv',index=False)

In [43]:
speeches1 = pd.read_csv('speeches1.csv')

Let's see if I can get anything more. Have updated the get_speeches function to return 'unable to scrape' when it errors.

In [69]:
speeches2 = pd.DataFrame([x for letter in speeches_dict.values() for x in letter])
speeches2.columns = ['title','speech']

In [70]:
speeches2.shape

(3080, 2)

In [72]:
speeches2[speeches2.speech=='unable to scrape']

,title,speech
1005,"Javid, Sajid – 2017 Speech to the Anne Frank T...",unable to scrape
1502,"Grieve, Dominic – 2017 Speech on Withdrawal fr...",unable to scrape
1652,"Blunkett, David – 2001 Speech to Police Superi...",unable to scrape


I looked up the three speeches that I couldn't scrape; the first two were broken links and the third was a pdf. I've got as much from this site as I can.

In [44]:
speeches1.iloc[0,1]

'\nBelow is the text of the speech made by Mark Isherwood on 10th January 2006 on fuel poverty.\nThe Chartered Institute of Housing Cymru states that it is reasonable to expect everyone to live in a safe, warm, affordable, and comfortable home. However, there are growing signs that current housing policy is failing to deliver that.\nWales has the oldest housing stock of any western European nation, with 8.5 per cent of the Welsh housing stock in home ownership classified as unfit. The Council of Mortgage Lenders reports that one in two of the poorest households live in their own homes and 72 per cent of Welsh properties that are officially designated as unfit are owner-occupied. The Joseph Rowntree Foundation tells us that three in five pensioners on low incomes in Wales are homeowners.\nNational Energy Action states that, across north Wales, 34,000 pensioners are on low incomes or claiming pension credit and 13.8 per cent of households in Wales live in fuel poverty. Each year, there a

Okay, a few things here. Firstly, I need to remove the line end characters. Secondly, before I do that I can use it to split out the description at the top of the speech (assuming it's consistent, will look into that). Thirdly, this text includes the Welsh translation, didn't realise I was going to get that.

In [45]:
descriptions = pd.DataFrame()
descriptions['description'] = speeches1.speech.map(lambda x: x.split('\n')[1])

In [46]:
descriptions['check'] = descriptions.description.map(lambda x: 1 if "Below is the text" in x else 0)

In [47]:
speeches1[descriptions.check==0]

,title,speech
7,"Umunna, Chuka – 2016 Speech on Gangs and Serio...",\nhttps://commons.wikimedia.org/w/index.php?cu...
8,"Zahawi, Nadhim – 2018 Speech at National Learn...",\n\nBelow is the text of the speech made by Na...
9,"Zahawi, Nadhim – 2018 Statement on Childcare",\n\nBelow is the text of the statement made by...
10,"Zambellas, George – 2016 Speech on the Royal Navy",\n\nBelow is the text of the speech made by Si...
14,"Yeo, Tim – 1983 Maiden Speech in the House of ...",\n\nBelow is the text of the maiden speech mad...
15,"Yeo, Tim – 2000 Speech to Conservative Party C...",\n\nBelow is the text of the speech made by Ti...
16,"Yeo, Tim – 2004 Speech on Patient Choice",\n\nBelow is the text of the speech by Tim Yeo...
22,"Neave, Airey – 1953 Maiden Speech in the House...",\n\nBelow is the text of the maiden speech mad...
23,"Neave, Airey – 1973 Speech on Science and Tech...",\n\nBelow is the text of the speech made by Ai...
24,"Nesbitt, Mike – 2015 Speech in Fivemiletown",\n\nBelow is the text of the speech made by Mi...


Turns out a lot of the descriptions have two line breaks before them, not one. Will update the map.

In [48]:
descriptions2 = pd.DataFrame()
descriptions2['description'] = speeches1.speech.map(lambda x: x.split('\n')[1] if x.split('\n')[1] else x.split('\n')[2])
descriptions2['check'] = descriptions2.description.map(lambda x: 1 if "Below is the text" in x else 0)

In [49]:
speeches1[descriptions2.check==0]

,title,speech
7,"Umunna, Chuka – 2016 Speech on Gangs and Serio...",\nhttps://commons.wikimedia.org/w/index.php?cu...
59,"Vaizey, Ed – 2013 Speech to the Oxford Media C...",\n\nThe below speech was made by Ed Vaizey on ...
60,"Vaizey, Ed – 2013 Speech to Cable Congress",\n\nThe below speech was made by the Parliamen...
70,"Villiers, Theresa – 2010 Speech on the Railways",\nTheresa Villiers\nBelow is the text of the s...
71,"Villiers, Theresa – 2011 Speech to Westminster...",\nTheresa Villiers\nBelow is the text of the s...
72,"Villiers, Theresa – 2013 Speech to Alliance Pa...",\nTheresa Villiers\nBelow is the text of the s...
73,"Villiers, Theresa – 2013 Speech to the British...",\nTheresa Villiers\nBelow is the text of the s...
74,"Villiers, Theresa – 2013 Speech to the Spirit ...",\nTheresa Villiers\nBelow is the text of the s...
75,"Villiers, Theresa – 2013 Speech on Investing i...",\nTheresa Villiers\nBelow is the text of the s...
76,"Villiers, Theresa – 2013 Speech on Keeping Nor...",\nTheresa Villiers\nBelow is the text of the s...


This isn't as clear cut as I thought. What I'll do is split the text then find the index where it says "Below is the text" or "Below is the speech" or "The below speech" and split at that point.

In [50]:
indicators = ['Below is the text',
              'Below is the speech',
              'The below speech',
              'The speech below',
              'Below is the Hansard record',
              'Below is a part of the speech',
              'This is the text',
              'This speech was made',
              'Speech made',
              'Speech given',
              'Below is the statement',
              'Below is the transcript',
              'Below is a transcript',
              'Below is a text',
              'Below is the Q&A',
              'Below is the Passover message',
              'Below is the Christmas message',
              'Below is the 2013 Christmas message',
              'This speech was given',
              'Transcript of speech',
              'Transcript of press conference']

def find_header(text):
    text_split = text.split('\n')
    for indicator in indicators:
        for i, subtext in enumerate(text_split):
            if indicator in subtext:
                return i
    return -1

def get_header(text):
    header_index = find_header(text)
    if  header_index > -1:
        return text.split('\n')[header_index]
    else:
        return 'No header'

def get_text(text):
    header_index = find_header(text)
#     I'm going to return the speech still separated by \n so that I can split out further features later.
    return '\n'.join(text.split('\n')[header_index+1:])
    

In [51]:
descriptions3 = pd.DataFrame()
descriptions3['description'] = speeches1.speech.map(lambda x: get_header(x))
descriptions3['text'] = speeches1.speech.map(lambda x: get_text(x))

In [52]:
descriptions3[descriptions3.description=='No header']

,description,text
217,No header,\n\nMy Lords and Members of the House of Commo...
232,No header,\nOne of the features of growing old is a heig...
424,No header,\n\nI’d like to thank Jill Kirby and the Centr...
485,No header,"\n\nMr Deputy Speaker, the core purpose of thi..."
646,No header,"\nEnoch Powell, referring to the comments made..."
686,No header,"\nThis session is about the local environment,..."
1121,No header,\n\nConvener – fellow Scots.\nThis is a speech...
1128,No header,"\nMy Lords, I trust that your Lordships will a..."
1149,No header,"\nWith permission, Mr Speaker, I would like to..."
1187,No header,"\nIain Duncan Smith\nAs I said last night, out..."


That worked well, only about 30 of ~3000 don't have headers. I'll keep the function and apply it directly to the dataframe later. I'll probably use a similar approach to take out Welsh words at the same time; I can remove them when vectorising for the NLP, but if I'm going to do a unique word count then I'll have to do it separately.<p>

In [53]:
unique_welsh = [' yn ',
                ' yr ',
                ' ydym ']

def find_welsh(text):
    text_split = text.split('\n')
    for word in unique_welsh:
        for i, subtext in enumerate(text_split):
            if word in subtext:
                return i
    return -1

In [54]:
welsh_speeches = speeches1[(speeches1.speech.str.contains(' yn '))|
                           (speeches1.speech.str.contains(' yr '))|
                           (speeches1.speech.str.contains(' ydym '))]

In [55]:
welsh_speeches['welsh_start'] = welsh_speeches.speech.map(lambda x: find_welsh(x))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [56]:
welsh_speeches.reset_index()

,index,title,speech,welsh_start
0,0,"Isherwood, Mark – 2006 Speech on Fuel Poverty",\nBelow is the text of the speech made by Mark...,18
1,144,"Ramsay, Nick – 2008 Speech on Local Government...",\nBelow is the text of a speech made by Nick R...,17
2,393,"Wood, Leanne – 2013 Speech to Plaid Cymru Conf...",\nBelow is the text of the speech made by the ...,36
3,762,"Lake, Ben – 2017 Maiden Speech in the House of...",\nBelow is the text of the maiden speech made ...,23
4,807,"Lewis, Brandon – 2017 Speech to CBI Wales",\n\nBelow is the text of the speech made by Br...,88
5,1102,"Jones, Carwyn – 2013 Speech to Labour Party Co...",\nBelow is the text of the speech made by Carw...,81
6,2205,"Cairns, Alun – 2015 Speech on the Welsh Language",\n\nBelow is the text of the speech made by Al...,46
7,2208,"Cairns, Alun – 2016 Speech to National Assembl...",\n\nBelow is the text of the speech made by Al...,3
8,2504,"Clwyd, Ann – 2018 Speech on the NHS Complaints...",\nBelow is the text of the speech made by Ann ...,7
9,2571,"Crabb, Stephen – 2015 Speech on Wales in a Cha...",\nStephen Crabb\nBelow is the text of the spee...,4


In [57]:
welsh_speeches.iloc[2,1]

'\nBelow is the text of the speech made by the Leader of Plaid Cymru, Leanne Wood, made to the party’s annual conference on 4th March 2013.\nConference,\nIt was a pleasure to travel through Wales yesterday to reach Ynys Mon, to cross the bridge to reach the apex of our country ready for this most auspicious day, the 1st of March.\nYnys Mon, Mam Cymru – is special to Wales.\nThis is the county of so many Welsh firsts –\n– the first comprehensive school was set up here, a great symbol of the best in our creative tradition,\n– our first woman MP in Megan Lloyd George,\n– the first branch of the Women’s Institute.\nIt is a fitting place for the first woman leader of Plaid Cymru to pay tribute to the first Plaid Cymru minister in the history of our party.\nI want to say Thank you to Ieuan for his tremendous service to Ynys Mon and to Wales for more than a quarter of a century.\nIn 1987, his victory here as an MP will be remembered as a ray of light in a dismal decade for Plaid Cymru.\nA bre

I thought that what we'd get would be speeches with the English version first and the Welsh version after, but actually we've got Welsh paragraphs embedded in English ones. I'll write a function that splits by \n then identifies any paragraphs with words starting 'yn', 'yd', 'ym', 'yw', or 'yr', the word 'ni', and words starting 'mwy' or 'hwy' and returns a string of just the other paragraphs. This might not be exhaustive but I think it will do enough of a job to remove most of the Welsh paragraphs without touching the English ones.

In [58]:
welsh_ident = re.compile(r"( y[ndmwr]| y | ni[^A-Za-z]| mwy| hwy)")

def return_english(text):
    text_split = text.split('\n')
    welsh_paras = []
    for i, para in enumerate(text_split):
        if re.findall(welsh_ident, para): welsh_paras.append(i)
    english_paras = [para for i,para in enumerate(text_split) if i not in welsh_paras]
#     for consistency still return joined by \n
    return '\n'.join(english_paras)

In [59]:
return_english(welsh_speeches.iloc[2,1])

'\nBelow is the text of the speech made by the Leader of Plaid Cymru, Leanne Wood, made to the party’s annual conference on 4th March 2013.\nConference,\nIt was a pleasure to travel through Wales yesterday to reach Ynys Mon, to cross the bridge to reach the apex of our country ready for this most auspicious day, the 1st of March.\nYnys Mon, Mam Cymru – is special to Wales.\nThis is the county of so many Welsh firsts –\n– the first comprehensive school was set up here, a great symbol of the best in our creative tradition,\n– our first woman MP in Megan Lloyd George,\n– the first branch of the Women’s Institute.\nIt is a fitting place for the first woman leader of Plaid Cymru to pay tribute to the first Plaid Cymru minister in the history of our party.\nI want to say Thank you to Ieuan for his tremendous service to Ynys Mon and to Wales for more than a quarter of a century.\nIn 1987, his victory here as an MP will be remembered as a ray of light in a dismal decade for Plaid Cymru.\nA bre

In [60]:
return_english(welsh_speeches.iloc[3,1])

'\nBelow is the text of the maiden speech made by Ben Lake, the Plaid Cymru MP for Ceredigion, in the House of Commons on 13 July 2017.\nDiolch, Madam Deputy Speaker. Thank you for affording me the opportunity to make my maiden speech this afternoon.\nIt is a pleasure to follow the hon. Member for Heywood and Middleton (Liz McInnes) and, in particular, the hon. Members for Glasgow North East (Mr Sweeney) and for Bedford (Mohammad Yasin), who both made excellent maiden speeches. Indeed, they set an exacting standard. They spoke from the heart and I have no doubt that they will be a credit to their party, their constituencies and this House.\nI welcome the opportunity to remember the third battle of Ypres in the House and to commemorate the first world war. As the years go by, it becomes increasingly important that we remember the conflict and especially the sacrifice of all those who lost their lives. We must ensure that we learn the lessons of the past and strive never again to subject

Next I need to identify any special characters:


In [61]:
# use the fact that regex is based on ascii characters to pull out everything that's not betwwen 32 (space) and 122 (z)
special_ident = re.compile(r"[^ -z]")
special_chars=[]
for i, row in speeches1.iterrows():
    results = re.findall(special_ident,row['speech'])
    if results: special_chars.append(results)

In [62]:
unique_chars = []
for chars in special_chars:
    for char in chars:
        if char not in unique_chars:
            unique_chars.append(char)
unique_chars

['\n',
 '—',
 '\xa0',
 '’',
 'ô',
 'â',
 '–',
 '£',
 '‘',
 '“',
 '”',
 'é',
 '\u200b',
 '…',
 '½',
 '°',
 'á',
 'í',
 '\u2003',
 '\u200e',
 'ï',
 '₂',
 'ê',
 'û',
 'è',
 'ü',
 'Æ',
 '\xad',
 '~',
 '•',
 '·',
 '−',
 '⅓',
 '¼',
 '€',
 'ã',
 'ö',
 '¾',
 'Ç',
 '§',
 '\x9d',
 'Ã',
 '±',
 '|',
 '\x80',
 '\x94',
 '\x9c',
 '\x99',
 'œ',
 'ß',
 'ä',
 'ç',
 'ﬁ',
 '‑',
 'ó',
 '´',
 '\u200a',
 'ë',
 'à',
 '�',
 'ñ',
 '˚',
 '‐',
 '\ufdd3',
 'Â',
 '\u202f',
 '′',
 '尊',
 '敬',
 '的',
 '刘',
 '晓',
 '明',
 '大',
 '使',
 '各',
 '位',
 '来',
 '宾',
 '家',
 '晚',
 '上',
 '好',
 '！',
 '恭',
 '祝',
 '中',
 '国',
 '朋',
 '友',
 '狗',
 '年',
 '吉',
 '‒',
 'ń',
 'В',
 'м',
 'е',
 'с',
 'т',
 'ы',
 'и',
 'л',
 'ь',
 'н',
 'ŵ',
 'ă',
 '\x96',
 'É',
 'ğ',
 '\x13',
 '‟',
 'ō',
 'ı',
 'î',
 '″',
 '\x91',
 '\x92',
 'å',
 'Ü']

If we remove all of these characters we're going to lose words with accented vowels. I think this is worth doing to ensure we've got clean data; it doesn't matter if a few words get mangled if it means we can strip out all of the weird characters. Also I think a few of them will be Welsh so will be removed anyway. Need a function to replace all of these characters with a space as the most common occurance will be the \n.

In [63]:
special_ident = re.compile(r"[^ -z’]")

def remove_specials(text):
    clean_text = re.sub(special_ident, ' ', text).strip()
    while '  ' in clean_text:
        clean_text = clean_text.replace('  ',' ')
    return clean_text

In [64]:
test_text = "don't (say) hi"

In [65]:
remove_specials(test_text)

"don't (say) hi"

In [66]:
speeches1.iloc[0,1]

'\nBelow is the text of the speech made by Mark Isherwood on 10th January 2006 on fuel poverty.\nThe Chartered Institute of Housing Cymru states that it is reasonable to expect everyone to live in a safe, warm, affordable, and comfortable home. However, there are growing signs that current housing policy is failing to deliver that.\nWales has the oldest housing stock of any western European nation, with 8.5 per cent of the Welsh housing stock in home ownership classified as unfit. The Council of Mortgage Lenders reports that one in two of the poorest households live in their own homes and 72 per cent of Welsh properties that are officially designated as unfit are owner-occupied. The Joseph Rowntree Foundation tells us that three in five pensioners on low incomes in Wales are homeowners.\nNational Energy Action states that, across north Wales, 34,000 pensioners are on low incomes or claiming pension credit and 13.8 per cent of households in Wales live in fuel poverty. Each year, there a

In [67]:
remove_specials(speeches1.iloc[0,1])

'Below is the text of the speech made by Mark Isherwood on 10th January 2006 on fuel poverty. The Chartered Institute of Housing Cymru states that it is reasonable to expect everyone to live in a safe, warm, affordable, and comfortable home. However, there are growing signs that current housing policy is failing to deliver that. Wales has the oldest housing stock of any western European nation, with 8.5 per cent of the Welsh housing stock in home ownership classified as unfit. The Council of Mortgage Lenders reports that one in two of the poorest households live in their own homes and 72 per cent of Welsh properties that are officially designated as unfit are owner-occupied. The Joseph Rowntree Foundation tells us that three in five pensioners on low incomes in Wales are homeowners. National Energy Action states that, across north Wales, 34,000 pensioners are on low incomes or claiming pension credit and 13.8 per cent of households in Wales live in fuel poverty. Each year, there are ov

In [68]:
remove_specials(return_english(speeches1.iloc[0,1]))

'Below is the text of the speech made by Mark Isherwood on 10th January 2006 on fuel poverty. The Chartered Institute of Housing Cymru states that it is reasonable to expect everyone to live in a safe, warm, affordable, and comfortable home. However, there are growing signs that current housing policy is failing to deliver that. Wales has the oldest housing stock of any western European nation, with 8.5 per cent of the Welsh housing stock in home ownership classified as unfit. The Council of Mortgage Lenders reports that one in two of the poorest households live in their own homes and 72 per cent of Welsh properties that are officially designated as unfit are owner-occupied. The Joseph Rowntree Foundation tells us that three in five pensioners on low incomes in Wales are homeowners. National Energy Action states that, across north Wales, 34,000 pensioners are on low incomes or claiming pension credit and 13.8 per cent of households in Wales live in fuel poverty. Each year, there are ov

Time to clean up the whole thing:

In [69]:
uk_pol_clean = speeches1.copy()

In [70]:
uk_pol_clean.columns = ['title','raw_speech']

In [71]:
uk_pol_clean['description'] = uk_pol_clean.raw_speech.map(lambda x: get_header(x))
uk_pol_clean['text'] = uk_pol_clean.raw_speech.map(lambda x: get_text(x))

In [72]:
uk_pol_clean['text'] = uk_pol_clean.text.map(lambda x: remove_specials(return_english(x)))

In [73]:
uk_pol_clean.drop('raw_speech',axis=1,inplace=True)

In [74]:
uk_pol_clean.to_csv('uk_pol_clean.csv',index=False)

In [75]:
uk_pol_clean = pd.read_csv('uk_pol_clean.csv')

Now need to extract a bit more data like MP name etc.

In [76]:
uk_pol_clean.head(1)

,title,description,text
0,"Isherwood, Mark – 2006 Speech on Fuel Poverty",Below is the text of the speech made by Mark I...,The Chartered Institute of Housing Cymru state...


In [77]:
test = pd.DataFrame(uk_pol_clean.title.map(lambda x: x.split('–')[0]))

In [78]:
test.columns = ['longname']

In [79]:
test['spaces'] = test.longname.map(lambda x: x.count(' '))

In [80]:
test[(test.longname.str.contains('20'))|(test.longname.str.contains('19'))|test.longname.str.contains('18')]

,longname,spaces
63,"Vara, Shailesh -2016 Speech on Family Justice",6
603,"Peel, Robert- 1846 Corn Law Statement",5
1489,"Greening, Justine- 2017 Statement on National ...",9
1791,"Bruce, Fiona -2016 Speech on Human Rights in B...",8


Some messy data here but not too hard to capture.

In [81]:
def get_speaker(text):
    longname = text.split('–')[0]
    if '20' in longname or '19' in longname or '18' in longname:
        name = ('{} {}').format(longname.split(' ')[1].strip().replace('-',''),
                                longname.split(' ')[0].strip().replace(',',''))
    elif ',' in longname:
        name = ('{} {}').format(longname.split(',')[1].strip(),
                                longname.split(',')[0].strip())
    else:
        name = longname.strip()
    return name

In [82]:
uk_pol_clean['speaker'] = uk_pol_clean.title.map(lambda x: get_speaker(x))

In [83]:
uk_pol_clean.speaker.value_counts()[:10]

David Cameron         186
Theresa May           158
Gordon Brown           67
Matthew Hancock        62
George Osborne         58
William Hague          53
Ed Miliband            49
Michael Gove           49
Queen Elizabeth II     45
Sajid Javid            44
Name: speaker, dtype: int64

Pull the year and subject from the title:

In [84]:
year_ident = re.compile(r'[0-9][0-9][0-9][0-9]')
        
def get_year(text):
    result = re.findall(year_ident, text)
    if result:
        year = int(result[0])
    else:
        year = 0
    return year

subject_prefixes = ['Speech in Tribute to ','Tribute to ','Speech on ','Speech to ','Speech about ',
                    'Speech following ','Speech after ','Speech After ',
                    'Speech against ','Statement on ','Statement following ','Statement after ']

def get_subject(title, year):
    if year > 0:
        subject = title.split(str(year))[1].strip().replace(' the ',' ')
    else:
        subject = title.strip()
    if 'Maiden Speech' not in subject:
        for prefix in subject_prefixes:
            subject = subject.replace(prefix,'')
    else:
        subject = 'Maiden Speech'
    return subject

In [85]:
uk_pol_clean['year'] = uk_pol_clean.title.map(lambda x: get_year(x))
uk_pol_clean['year'] = uk_pol_clean.year.astype(int)

In [86]:
uk_pol_clean['subject'] = uk_pol_clean.apply(lambda x: get_subject(x['title'],x['year']), axis=1)

In [87]:
uk_pol_clean.subject.value_counts()[:10]

Maiden Speech                        158
Labour Party Conference              124
Conservative Party Conference         68
Queen’s Speech                        51
TUC Conference                        25
Speech at Labour Party Conference     23
Syrian Air Strikes                    19
Brexit                                15
King’s Speech                         14
Mansion House Speech                  14
Name: subject, dtype: int64

In [88]:
test_desc = uk_pol_clean.iloc[100,1]
test_desc

'Below is the text of the speech made by Charles Kennedy, the then Leader of the Liberal Democrats, on 29 January 2001.'

In [89]:
full_date = re.compile(r'[0-9]{1,2}[stndrdth]{0,2} [A-Z][a-z]+ [0-9]{4}')

def find_full_date(text):
    dates = re.findall(full_date, text)
    if dates:
        date = dates[0]
    else:
        date = np.nan
    return date

In [90]:
uk_pol_clean['date'] = uk_pol_clean.description.map(lambda x: find_full_date(x))

In [91]:
uk_pol_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3077 entries, 0 to 3076
Data columns (total 7 columns):
title          3077 non-null object
description    3077 non-null object
text           3076 non-null object
speaker        3077 non-null object
year           3077 non-null int64
subject        3077 non-null object
date           2878 non-null object
dtypes: int64(1), object(6)
memory usage: 168.4+ KB


In [92]:
uk_pol_clean.date = pd.to_datetime(uk_pol_clean.date)

In [93]:
uk_pol_clean.date[:10]

0   2006-01-10
1   2011-12-02
2   2012-01-12
3   2012-06-26
4   2012-05-28
5          NaT
6          NaT
7   2016-03-03
8   2018-02-27
9   2018-07-02
Name: date, dtype: datetime64[ns]

In [94]:
uk_pol_clean.to_csv('uk_pol_clean.csv', index=False)